In [515]:
using Zygote
using ITensors
using Random

In [516]:
function angle_encoder(x::Float64) 
    """Function to convert normalised time series to an angle encoding."""
    @assert x <= 1.0 && x >= 0.0 "Data points must be rescaled between 1 and 0 before encoding using the angle encoder."
    s1 = exp(1im * (3π/2) * x) * cospi(0.5 * x)
    s2 = exp(-1im * (3π/2) * x) * sinpi(0.5 * x)
    return [s1, s2]
 
end

angle_encoder (generic function with 1 method)

In [517]:
function normalised_data_to_product_state(sample::Vector, site_indices::Vector{Index{Int64}})
    """Function to convert a single normalised sample to a product state
    with local dimension 2, as specified by the feature map."""

    @assert length(sample) == length(site_indices) "Mismatch between number of sites and sample length."

    product_state = MPS([ITensor(angle_encoder(sample[i]), site_indices[i]) for i in eachindex(site_indices)])

    return product_state

end

normalised_data_to_product_state (generic function with 1 method)

In [518]:
s = siteinds("S=1/2", 10);
mps = randomMPS(ComplexF64, s; linkdims=5);

In [519]:
sample = rand(10)

10-element Vector{Float64}:
 0.5728769140695948
 0.3488862895084386
 0.5997585730834746
 0.46858811619538765
 0.5953328064858171
 0.9050071477944349
 0.028502352500627826
 0.10908886624638714
 0.5987456154072579
 0.19184520680855666

In [520]:
ps = normalised_data_to_product_state(sample, s)

MPS
[1] ((dim=2|id=347|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=668|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=444|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=136|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=861|"S=1/2,Site,n=5"),)
[6] ((dim=2|id=430|"S=1/2,Site,n=6"),)
[7] ((dim=2|id=754|"S=1/2,Site,n=7"),)
[8] ((dim=2|id=420|"S=1/2,Site,n=8"),)
[9] ((dim=2|id=701|"S=1/2,Site,n=9"),)
[10] ((dim=2|id=239|"S=1/2,Site,n=10"),)


In [521]:
function raw_overlap(ps, mps)
    res = conj(ps[1]) * mps[1]
    for i in 2:length(mps)
        res *= conj(ps[i]) * mps[i]
    end
    return res[]
end

raw_overlap (generic function with 1 method)

In [522]:
bt = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=347|"S=1/2,Site,n=1") (dim=2|id=668|"S=1/2,Site,n=2") (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [525]:
function loss(bt, mps, ps)
    phi_tilde = conj(ps[1]) * conj(ps[2])
    for i in 3:10
        phi_tilde *= mps[i] * conj(ps[i])
    end
    yhat = phi_tilde * bt
    p = norm(yhat[])^2
    return -log(p)
end

loss (generic function with 1 method)

In [538]:
l = x -> loss(x, mps, ps)

#91 (generic function with 1 method)

In [539]:
f, (g,) = withgradient(l, bt)

(val = 8.85801398350347, grad = (ITensor ord=3
Dim 1: (dim=2|id=347|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=668|"S=1/2,Site,n=2")
Dim 3: (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×5
[:, :, 1] =
 0.8786511812478723 - 1.2441974973547811im   …  -0.41964944316446007 + 0.8296451015251627im
 1.9139697506485613 - 0.13799517256384375im      -1.1434498448528752 + 0.25396630069413717im

[:, :, 2] =
 -0.6794419762505125 + 0.8006409544375885im   …  0.3389015285206844 - 0.5440427494300614im
 -1.3227336391391669 - 0.02228370395221435im     0.8007207762810589 - 0.1044707574625175im

[:, :, 3] =
 -3.974554551017032 + 1.0349836874527731im  …  2.307766163529125 - 0.9793228249629458im
 -4.183362773767195 - 3.0450453370098027im     2.797605521061895 + 1.4657909330868217im

[:, :, 4] =
 -0.14843414359039986 - 0.7719471458352322im  …   0.15845417312418675 + 0.4529076201259473im
   0.6334221264228121 - 0.761278287425299im      -0.31462172711404757 + 0.5161707148253162im

[:, :,

Analytical gradient

In [540]:
phi_tilde = conj(ps[1]) * conj(ps[2])
for i in 3:10
    phi_tilde *= mps[i] * conj(ps[i])
end

In [541]:
g_analytical = conj(phi_tilde)/conj(yhat)

ITensor ord=3 (dim=2|id=347|"S=1/2,Site,n=1") (dim=2|id=668|"S=1/2,Site,n=2") (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [534]:
@show g

g = ITensor ord=3
Dim 1: (dim=2|id=347|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=668|"S=1/2,Site,n=2")
Dim 3: (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×5
[:, :, 1] =
 0.8786511812478723 - 1.2441974973547811im   -0.41964944316446007 + 0.8296451015251627im
 1.9139697506485613 - 0.13799517256384375im   -1.1434498448528752 + 0.25396630069413717im

[:, :, 2] =
 -0.6794419762505125 + 0.8006409544375885im   0.3389015285206844 - 0.5440427494300614im
 -1.3227336391391669 - 0.02228370395221435im  0.8007207762810589 - 0.1044707574625175im

[:, :, 3] =
 -3.974554551017032 + 1.0349836874527731im  2.307766163529125 - 0.9793228249629458im
 -4.183362773767195 - 3.0450453370098027im  2.797605521061895 + 1.4657909330868217im

[:, :, 4] =
 -0.14843414359039986 - 0.7719471458352322im   0.15845417312418675 + 0.4529076201259473im
   0.6334221264228121 - 0.761278287425299im   -0.31462172711404757 + 0.5161707148253162im

[:, :, 5] =
 -0.8269253513853413 - 1.7493763007494585im   0

ITensor ord=3 (dim=2|id=347|"S=1/2,Site,n=1") (dim=2|id=668|"S=1/2,Site,n=2") (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [535]:
@show g_analytical

g_analytical = ITensor ord=3
Dim 1: (dim=2|id=347|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=668|"S=1/2,Site,n=2")
Dim 3: (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×5
[:, :, 1] =
  0.872466753141113 + 0.08047509641817902im   -0.5340146305371377 + 0.029188797862906058im
 0.6185852353938848 + 0.9142113719187939im   -0.45503880744939196 - 0.49689824299534385im

[:, :, 2] =
  -0.5941889195012862 - 0.10861410447688548im  0.3684849690842469 + 0.01261251177116235im
 -0.36886761175785054 - 0.6656032335314274im   0.2820821854404508 + 0.36903924470528143im

[:, :, 3] =
 -1.6575420609665077 - 1.6834592827617252im    1.150995390912246 + 0.8687824022543147im
  0.3158118233975521 - 2.9595647466382746im  0.07315328788595292 + 1.8152921227137608im

[:, :, 4] =
 0.3420704496564142 - 0.29572631335911664im  -0.18019446139688636 + 0.20907154247046286im
 0.5613521901315077 + 0.09698695335016333im   -0.3476198702609363 - 0.008518965163634203im

[:, :, 5] =
 0.6342795909612343 - 0

ITensor ord=3 (dim=2|id=347|"S=1/2,Site,n=1") (dim=2|id=668|"S=1/2,Site,n=2") (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [377]:
# for i in 1:100
#     f, (g,) = withgradient(l, bt_old)
#     bt_new = bt_old - 0.8 * g
#     println(f)
#     bt_old = bt_new
# end

0.9747101482636243
0.9494859734123181
0.9254879825864393
0.9026018258464243
0.8807284932305783
0.8597816814119945
0.8396857032719909
0.8203738111803388
0.8017868393257689
0.7838720948104204
0.7665824446683752
0.7498755586301509
0.7337132767606614
0.7180610780171822
0.7028876309740216
0.6881644119082481
0.6738653784656469
0.6599666894640668
0.6464464632132463
0.6332845681608003
0.6204624408055389
0.6079629267202402
0.5957701412479696
0.5838693470181279
0.5722468459003637
0.5608898833992414
0.5497865638078325
0.5389257746980178
0.5282971195400475
0.5178908574223855
0.5076978489917734
0.4977095078582302
0.48791775681465593
0.4783149883092854
0.46889402868431584
0.45964810575781884
0.450570819380475
0.4416561146452089
0.4328982574677675
0.4242918122906346
0.41583162169237314
0.4075127877101261
0.3993306547053042
0.3912807936218364
0.38335898750325226
0.37556121814962723
0.3678836538083322
0.36032263780387497
0.35287467802210487
0.34553643717283394
0.3383047237626958
0.33117648371693487
0.3

In [380]:
phi_tilde = conj(ps[1]) * conj(ps[2])
for i in 3:10
    phi_tilde *= mps[i] * conj(ps[i])
end
yhat = phi_tilde * bt_old

ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

-------

Alt. formulation

In [523]:
function loss2(bt, mps, ps)
    phi_tilde = conj(ps[1]) * conj(ps[2])
    for i in 3:10
        phi_tilde *= mps[i] * conj(ps[i])
    end
    yhat = phi_tilde * bt
    p = norm(yhat[])^2
    return -log(p)
end

loss2 (generic function with 1 method)

In [524]:
bt = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=347|"S=1/2,Site,n=1") (dim=2|id=668|"S=1/2,Site,n=2") (dim=5|id=261|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [480]:
l2 = x -> loss2(x, mps, ps)

#79 (generic function with 1 method)

In [481]:
l2(bt)

6.789787517503335

In [482]:
gradient(l2, bt)

(ITensor ord=3
Dim 1: (dim=2|id=718|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=732|"S=1/2,Site,n=2")
Dim 3: (dim=5|id=432|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×5
[:, :, 1] =
 -0.05414210766480898 + 0.0594908679666256im  …  0.17746053947913137 + 0.2109003899501478im
   1.3590547832759137 - 0.7880791112726404im      -2.059346083334616 - 4.973661666926607im

[:, :, 2] =
 0.0019243420069964459 - 0.013965008118887097im  …  -0.04655544407735828 - 0.012877784258328147im
   -0.1182351308082307 + 0.24863890317183862im        0.7907567184779841 + 0.5144802453209532im

[:, :, 3] =
 0.01719848219978476 - 0.03201931767150516im  …  -0.10093664313173749 - 0.07295264064605618im
 -0.5091424036824719 + 0.49463253366578985im         1.44869659446542 + 1.9538358884821951im

[:, :, 4] =
 -0.008303274108484755 - 0.009471652490615887im  …  -0.03593947585002528 + 0.023898916228253887im
   0.09869390571110129 + 0.22533743850778318im        0.8101351700168455 - 0.23285835575650804im

[:, :, 5

In [506]:
phi_tilde = conj(ps[1]) * conj(ps[2])
for i in 3:10
    phi_tilde *= mps[i] * conj(ps[i])
end
yhat = phi_tilde * bt

ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [507]:
@show -2*conj(phi_tilde)/conj(yhat)

(-2 * conj(phi_tilde)) / conj(yhat) = ITensor ord=3
Dim 1: (dim=2|id=719|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=828|"S=1/2,Site,n=2")
Dim 3: (dim=5|id=84|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×5
[:, :, 1] =
 1.3338437420061697 + 2.868811592181863im    0.5220354065231287 - 1.1511926329302968im
  9.043048354887542 + 0.11839265132459656im  -2.318788670926665 - 2.771177428160767im

[:, :, 2] =
 0.09782524082526582 - 1.0692796979774604im  -0.3495000900870513 + 0.2487790294221231im
   -2.63337810074475 - 1.5768718147630123im  0.20784412749922854 + 1.2085950230431786im

[:, :, 3] =
 -2.3123494152892587 + 2.778483084197521im  1.4440733909056411 - 0.022752534511623675im
   4.292632123643136 + 9.399480998751487im  1.7306280453219194 - 3.7482879238786277im

[:, :, 4] =
  -1.3633653712853777 + 0.48951042648966625im  0.5033368469429462 + 0.2856851613297097im
 -0.42820757742226506 + 4.118698993664228im     1.359603691098526 - 0.9426821734817034im

[:, :, 5] =
 -2.02060197167615

ITensor ord=3 (dim=2|id=719|"S=1/2,Site,n=1") (dim=2|id=828|"S=1/2,Site,n=2") (dim=5|id=84|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

checks out

In [508]:
g = -2*conj(phi_tilde)/conj(yhat)

ITensor ord=3 (dim=2|id=719|"S=1/2,Site,n=1") (dim=2|id=828|"S=1/2,Site,n=2") (dim=5|id=84|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [486]:
bt_new = bt - 0.4 * g

ITensor ord=3 (dim=2|id=718|"S=1/2,Site,n=1") (dim=2|id=732|"S=1/2,Site,n=2") (dim=5|id=432|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [487]:
l2(bt_new)

2.3705090817801664

In [491]:
bt_old = bt

ITensor ord=3 (dim=2|id=718|"S=1/2,Site,n=1") (dim=2|id=732|"S=1/2,Site,n=2") (dim=5|id=432|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [492]:
for i in 1:15
    f, (g,) = withgradient(l2, bt_old)
    bt_new = bt_old - 0.4 * g
    bt_old = bt_new 
    println(f)
end

6.789787517503335
2.3705090817801664
2.1840798107752595
2.0281574555554274
1.8940092592775732
1.7762162813743438
1.6711750231185483
1.5763635253822805
1.4899457435870969
1.4105423277661053
1.337090099139875
1.268751887818873
1.204856468143833
1.1448572614089865
1.088303167459855
